## Quantitative Methods 2:  Data Science and Visualisation
## Workshop 6: Advanced Dataframe Operations

Today we will be talking about and using **merging** and **joining** and performing some advanced Dataframe operations.

## Downloading the Data
Let's grab the data we will need this week from our course website and save it into our data folder. If you've not already created a data folder then do so using the following command. 

Don't worry if it generates an error, that means you've already got a data folder.

In [ ]:
!mkdir data

In [ ]:
!mkdir data/wk6
!curl https://s3.eu-west-2.amazonaws.com/qm2/wk6/veg.csv -o ./data/wk6/veg.csv
!curl https://s3.eu-west-2.amazonaws.com/qm2/wk6/ITF_GOODS_TRANSPORT_Data_fb8faed3-af67-457b-8387-8485f0ceea42.csv -o ./data/wk6/ITF_GOODS_TRANSPORT_Data_fb8faed3-af67-457b-8387-8485f0ceea42.csv
!curl https://s3.eu-west-2.amazonaws.com/qm2/wk6/AIR_GHG_Data_5f3eccd7-21db-4950-8390-2311d87476ee.csv -o ./data/wk6/AIR_GHG_Data_5f3eccd7-21db-4950-8390-2311d87476ee.csv

`--------------------------------`

Let's remind ourselves of the two types of join

In [ ]:
from IPython.display import Image
Image("https://s3.eu-west-2.amazonaws.com/qm2/wk1/inner_join.png")

In [ ]:
Image("https://s3.eu-west-2.amazonaws.com/qm2/wk6/left_outer_join.png")

We start with an example.

Let's create two Dataframes 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

In [ ]:
df1 = pd.DataFrame(np.random.randn(5, 5))

In [ ]:
df1

In [ ]:
df2 = pd.DataFrame(np.random.randn(3, 5))

In [ ]:
df2

Let's merge them by rows. concat does that for us.

In [ ]:
df3 = pd.concat([df1,df2])

In [ ]:
df3

What is the problem with the dataframe above? 

`-----------------------------------------`   
  
The solution is to set ignore_index to True.

In [ ]:
df3 = pd.concat([df1,df2], ignore_index=True)

In [ ]:
df3

`ignore_index` is very useful when we want a new DataFrame which only contains data from other DataFrames , but unrelated otherwise. 


Now we want to look at more comlex merge operations, which take in to account the data values.

In [ ]:
veg = {
    'vegetables': ['potato', 'mung bean', 'carrot', 'lettuce'],
    'species': ['S. tuberosum', 'V. radiata', 'D. carota', 'L. sativa'],
    'energy/100g (kJ)': [322, 334, 173, 55],
}

df = pd.DataFrame(veg)

In [ ]:
df3

In [ ]:
data_path = "./data/wk6/veg.csv"

df_veg = pd.read_csv(data_path)

In [ ]:
df_veg

You can quickly summarise data in a DataFrame column by using the `value_counts()` method

In [ ]:
df_veg['vegetables'].value_counts()

Note that the returned data is a `Series`, we can turn it in to a DataFrame quite easily using 'to_frame'.

In [ ]:
df_veg_counts  = df_veg['vegetables'].value_counts().to_frame(name='veg counts')

In [ ]:
df_veg_counts

In [ ]:
df

Joining the value counts to our categories

In [ ]:
df_joined = df.merge(
    df_veg_counts,
    how='inner',
    left_on='vegetables', right_index=True)

In [ ]:
df_joined

This is a complex operation, so let's break down exactly how we did it, beginning with the **innermost** operations

1. We used the `value_counts()` method to generate a `Series`
2. We converted the `Series` into a DataFrame using `to_frame()`
    - The resulting DataFrame uses the **left-most** column of values as its index
    - We renamed the value column to 'veg counts' using the `name` keyword.
3. We specified that we wished to perform an **inner join**
    - Joins are carried out using **keys**
    - Specifying an **inner** join means we are using only the key values that our DataFrames have in common
    - The `left_on` keyword lets us specify the column we want to use for our keys
    - The `right_index` keyword lets us specify that we wish to use the **index** from our right DatafFrame (the veg counts) as keys. 
4. Notice that although we have a value count of parnsips, there is no parsnip data in our joined DataFrame. This is because that data was discarded due to no matching key existing in the "left" DataFrame.

5. Pandas has **four** join strategies:
    - Left Join: use **only** keys from **left** DataFrame. SQL: [left outer join](http://goo.gl/JICveI)
    - Right Join: use **only** keys from **right** DataFrame. SQL: [right outer join](http://goo.gl/TrrHjQ)
    - Outer Join: use union of **keys from both** DataFrames. SQL: [full outer join](http://goo.gl/bVRqO8)
    - Inner Join: use **intersection of keys** from both DataFrames. SQL: [inner join](http://goo.gl/Cf1MF8)

The operations below show what effect each join strategy has:

In [ ]:
# note the NaN value for lettuce - there's no value count of it
df.merge(
    df_veg_counts,
    how='left',
    left_on='vegetables', right_index=True)

In [ ]:
# note the NaN values from our left dataframe
df.merge(
    df_veg_counts,
    how='right',
    left_on='vegetables', right_index=True)

In [ ]:
# note the NaN values from both left and right DataFrames
df.merge(
    df_veg_counts,
    how='outer',
    left_on='vegetables', right_index=True)

In [ ]:
# note no NaN values, only common keys are used to produce the result
df.merge(
    df_veg_counts,
    how='inner',
    left_on='vegetables', right_index=True)

Applying merging in real data - OECD transport and greenhouse gas emission data. 

In [ ]:
# Our previous file on population density.
data_path = "./data/wk6/ITF_GOODS_TRANSPORT_Data_fb8faed3-af67-457b-8387-8485f0ceea42.csv"
OECD_trs =  pd.read_csv(data_path)

# Data on greenhous gas emissions.
data_path = "./data/wk6/AIR_GHG_Data_5f3eccd7-21db-4950-8390-2311d87476ee.csv"
OECD_ghg = pd.read_csv(data_path)

OECD_trs.head()

In [ ]:
OECD_ghg.head()

In [ ]:
trs_years = pd.unique(OECD_trs['Year'])

In [ ]:
trs_years

In [ ]:
ghg_years = pd.unique(OECD_ghg['Year'])

In [ ]:
ghg_years

##The years seem not to coincide. 
##We will use merge to get an intersection of years, and to filter for a specific country. 


In [ ]:
Variable_types = pd.unique(OECD_trs['Variable'])

In [ ]:
Variable_types 

In [ ]:
Country = 'Spain'

# We filter the transport dataframe for a specific country.
OECD_trs_Country = OECD_trs[OECD_trs['Country'] == Country]

# the variable we are interested in is total freight.
OECD_trs_Country = OECD_trs_Country[OECD_trs_Country['Variable'] == 'Total inland freight in million tonne-km']

In [ ]:
OECD_trs_Country

In [ ]:
Variable_types = pd.unique(OECD_ghg['Variable'])

In [ ]:
Variable_types

We can merge the dataframes by both 'Year' and 'Country'. 

Both DataFrames contain columns called 'Year' and 'Country' , this enables to specify the names instead of specifying which column to join in each DataFrame. This is achieved by using the 'on' parameter. 

The 'Year' merge will ensure the intersection of years, and the 'Country' merge will ensure the result is intersected by the 'Country' variable since one dataframe was already filtered by it.

In [ ]:
# Resetting the DataFrame to contain all countries 
OECD_ghg_Country = pd.DataFrame()
OECD_ghg_Country = OECD_ghg[OECD_ghg['Variable'] == 'Total  emissions excluding LULUCF']
OECD_ghg_Country = OECD_ghg_Country[OECD_ghg_Country['Pollutant'] == 'Carbon dioxide']

# Merging by two variables.
OECD_trs_ghg_Country = OECD_trs_Country.merge(
    OECD_ghg_Country,
    how='inner',
    on=['Year','Country'])

In [ ]:
OECD_trs_ghg_Country

Notice that Pandas created a DataFrame with columns such as 'value_x' and 'value_y', this is because in the both the original DataFrames there was a column named 'value' , therefore in the merged DataFrame Python puts the 'value' for the left DataFrame in 'value_x' and 'value' of the right DataFrame in 'value_y'.

In [ ]:
# Creating a 'nice' DataFrame for plotting, etc.. 

OECD_trs_ghg_Filtered = pd.DataFrame()

OECD_trs_ghg_Filtered['Road freight'] = OECD_trs_ghg_Country['Value_x']
OECD_trs_ghg_Filtered['Carbon dioxide'] = OECD_trs_ghg_Country['Value_y']

OECD_trs_ghg_Filtered = OECD_trs_ghg_Filtered.set_index(OECD_trs_ghg_Country['Year'])

In [ ]:
OECD_trs_ghg_Filtered

In [ ]:
OECD_trs_ghg_Filtered.plot(legend = True)
ylabel('Absolute value')

In [ ]:
OECD_trs_ghg_Filtered_Pct = OECD_trs_ghg_Filtered.pct_change()

OECD_trs_ghg_Filtered_Pct = OECD_trs_ghg_Filtered_Pct.replace('NaN','0', regex=True).astype('float')
OECD_trs_ghg_Filtered_Pct
OECD_trs_ghg_Filtered_Pct.plot(legend = True)
ylabel('Percent change')

### Question :
Can you see any pattern in the data? Can you conclude anything?

## Exercise:

1. Choose two countries from the OECD, and choose a variable from each DataFrame that you would like to compare. 
2. Use merge to create the DataFrames for each country.
3. Plot absolute values, percent changes, means, medians. Compare the results, see if anything can be concluded. 
4. More advanced : Create an empty DataFrame whose index represents a range of years to investigate. (maybe a narrower range than the instersection of year ranges from each data set.) Use a merge operation to create the DataFrames for each country. 
